# 🔮 CSIRO Image2Biomass - 推論ノートブック (Kaggle)
📋 このノートブックの機能
✅ config.yaml と best_fold*.pth を読み込み
✅ test 画像を推論（画像入力のみ）
✅ （学習が log1p の場合）expm1 で元スケールへ戻す
✅ 提出形式（sample_id, target）の submission.csv を生成
🚀 使い方
Kaggle Notebook に以下2つを Add Data する

競技データセット（test.csv / sample_submission.csv / test画像）
学習済み成果物データセット（config.yaml / best_fold0.pth など）
このノートブックを実行

📝 注意事項
Kaggle は internet off のことが多いので、推論時は pretrained=False に固定
予測値は mass（g）なので、負値は 0 にクリップ
📦 ライブラリのインポート

In [1]:
# ========================
# Import libraries
# ========================
import os
import re
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import polars as pl
from PIL import Image
from typing import Union, Any

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import yaml
from tqdm.auto import tqdm

print("✅ ライブラリのインポート完了")

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

✅ ライブラリのインポート完了


/usr/local/lib/python3.12/dist-packages/albumentations/check_version.py:147: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


In [2]:
ARTIFACT_ROOTS = [
    "/kaggle/input/csiro-artifacts/exp009"
]

## 🔧 ユーティリティ関数¶

### 🌱 Seed固定関数などを実装

In [3]:
# ========== 汎用表示・ログ系 ==========
def sep(word: str, num: int = 80):
    print("=" * num)
    print(word)
    print("=" * num)

def show_df(df: Union[pd.DataFrame, pl.DataFrame], num: int = 3, show_tail: bool = False):
    """データフレームの概要を表示"""
    print(df.shape)
    display(df.head(num))
    if show_tail:
        display(df.tail(num))


# ========== ファイル操作 ==========
def glob_walk(root: Union[str, Path], pattern: str) -> list[Path]:
    """指定パターンにマッチするファイルパスを取得"""
    path = Path(root)
    return sorted(list(path.glob(pattern)))

# ========== seed固定 ==========
def seed_everything(seed: int = 1129) -> None:
    """
    再現性のためのseed固定。
    推論では必須ではないが、念のため設定。
    Args:
        seed (int): 乱数シード値
    """
    import random

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    print(f"🌱 Seed固定完了: {seed}")

### 📄 YAML読み込み関数

In [4]:
def _pick_state_dict(ckpt: Dict) -> Dict:
    """
    checkpoint dict から model state_dict を取り出す。
    
    Args:
        ckpt (Dict): チェックポイントの辞書
    
    Returns:
        Dict: モデルのstate_dict
    
    Raises:
        KeyError: state_dictが見つからない場合
    """
    for key in ["model_state_dict", "state_dict", "model"]:
        if key in ckpt and isinstance(ckpt[key], dict):
            return ckpt[key]

    # 直に state_dict っぽい場合
    if all(isinstance(k, str) for k in ckpt.keys()):
        return ckpt
    
    raise KeyError("❌ state_dict が見つかりません。checkpoint の保存形式を確認してください。")

In [5]:
def _load_yaml(path: Path) -> Dict:
    """
    YAMLファイルを読み込む。
    
    Args:
        path (Path): YAMLファイルのパス
    
    Returns:
        Dict: 設定情報の辞書
    """
    with open(path, "r") as f:
        cfg = yaml.safe_load(f)

    return cfg

### 📄 Testデータ前処理関数

In [6]:
def load_and_prepare_data(df: pd.DataFrame, is_test: bool = True) -> pd.DataFrame:
    """
    訓練/テストデータを読み込み、前処理を行います。
    
    Args:
        data_root (str): データディレクトリのルートパス
        is_test (bool): テストデータの場合True
    
    Returns:
        pd.DataFrame: 前処理済みのDataFrame
    """    
    if not is_test:
        # 訓練データを用いる場合はテストデータ風に整形
        df = df[["sample_id", "image_path", "target_name"]]

    # sample_idを分割
    df[['sample_id_prefix', 'sample_id_suffix']] = df.sample_id.str.split('__', expand=True)
    # target_nameごとに集約
    df['target'] = 0.0
    cols = ['sample_id_prefix', 'image_path']
    agg_df = df.groupby(cols).apply(
        lambda group: group.set_index('target_name').target
    )
    agg_df.reset_index(inplace=True)
    agg_df.columns.name = None
    return agg_df

### 🧠 モデル定義

In [7]:
# -*- coding: utf-8 -*-
"""
timm backbone regressor (画像のみ入力).

このクラスは名前は ConvNeXtRegressor ですが、実態は
「timm の任意バックボーンを回帰に使う」ための汎用モデルです。

✅ 対応例:
- convnext_small / convnext_base
- swin_tiny_patch4_window7_224 など（Swin）
- tf_efficientnetv2_s など（EfficientNetV2）

注意:
- timm のモデルによって create_model の引数対応が微妙に違うので、
  TypeError が出たら「引数を減らして再try」するフォールバックを入れています。
"""

from __future__ import annotations

from typing import Dict

import torch
import torch.nn as nn
import timm


_BACKBONE_ALIAS: Dict[str, str] = {
    # 使いやすいショート名 → timmの正式名
    "swin_tiny": "swin_tiny_patch4_window7_224",
    "efficientnetv2_s": "tf_efficientnetv2_s",
}


class ConvNeXtRegressor(nn.Module):
    """timmバックボーン + 線形ヘッドの回帰モデル。

    Args:
        backbone: timm のモデル名（例: convnext_small, swin_tiny_patch4_window7_224）
        pretrained: 事前学習重みを使うか
        num_targets: 出力ターゲット数（CSIROは 5）
        in_chans: 入力チャンネル（RGB=3）
        drop_rate: backbone 内の dropout（対応モデルのみ）
        drop_path_rate: backbone 内の stochastic depth（対応モデルのみ）
        head_dropout: head の dropout（最後の線形の前）
        global_pool: 特徴の集約方法（対応モデルのみ。通常 "avg"）
    """

    def __init__(
        self,
        backbone: str = "convnext_base",
        pretrained: bool = True,
        num_targets: int = 5,
        in_chans: int = 3,
        drop_rate: float = 0.0,
        drop_path_rate: float = 0.0,
        head_dropout: float = 0.0,
        global_pool: str = "avg",
    ) -> None:
        super().__init__()

        # alias 対応（swin_tiny など）
        backbone = _BACKBONE_ALIAS.get(backbone, backbone)
        self.backbone_name = backbone

        # -------------------------
        # backbone（特徴抽出器）
        # num_classes=0 で「分類headなし」の特徴抽出モード
        # -------------------------
        kwargs = dict(
            pretrained=pretrained,
            num_classes=0,
            in_chans=in_chans,
            drop_rate=drop_rate,
            drop_path_rate=drop_path_rate,
            global_pool=global_pool,
        )

        # timm のモデルによっては global_pool / drop_* を受け取れないことがあるので安全にフォールバック
        try:
            self.backbone = timm.create_model(backbone, **kwargs)
        except TypeError:
            # global_pool を外す
            kwargs2 = dict(kwargs)
            kwargs2.pop("global_pool", None)
            try:
                self.backbone = timm.create_model(backbone, **kwargs2)
            except TypeError:
                # drop系も外す
                kwargs3 = dict(kwargs2)
                kwargs3.pop("drop_rate", None)
                kwargs3.pop("drop_path_rate", None)
                self.backbone = timm.create_model(backbone, **kwargs3)

        feat_dim = getattr(self.backbone, "num_features", None)
        if feat_dim is None:
            raise RuntimeError(
                f"[ERROR] timm model '{backbone}' does not have 'num_features'. "
                "Please check timm version / model name."
            )

        self.head_dropout = nn.Dropout(p=float(head_dropout)) if head_dropout and head_dropout > 0 else nn.Identity()
        self.head = nn.Linear(int(feat_dim), int(num_targets))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward.

        Args:
            x: (B, C, H, W)

        Returns:
            y: (B, num_targets)
        """
        feat = self.backbone(x)

        # 念のため、モデルによって 3D/4D で返るケースを吸収
        # 例:
        # - CNN系: (B, C) or (B, C, H, W)
        # - Transformer系: (B, N, C)
        if feat.ndim == 4:
            feat = feat.mean(dim=(2, 3))
        elif feat.ndim == 3:
            feat = feat.mean(dim=1)

        feat = self.head_dropout(feat)
        out = self.head(feat)
        return out

## 🖼️ データセット & Transform

### 🔄 Transform定義

In [8]:
def build_infer_transform(
    img_size: int, 
    mean: List[float], 
    std: List[float]
) -> A.Compose:
    """
    推論用のtransform（最小構成）。
    
    Args:
        img_size (int): リサイズ後の画像サイズ
        mean (List[float]): 正規化の平均値
        std (List[float]): 正規化の標準偏差
    
    Returns:
        A.Compose: Albumentationsのtransform
    """
    return A.Compose(
        [
            A.Resize(img_size, img_size),
            A.Normalize(mean=mean, std=std),
            ToTensorV2(),
        ]
    )

### 📦 Dataset定義

In [9]:
class TestImageDataset(Dataset):
    """
    test画像（unique image_path）用Dataset。
    
    Args:
        df_unique (pd.DataFrame): ユニークな画像パスを含むDataFrame
        data_root (Path): データのルートディレクトリ
        transform (A.Compose): 適用するtransform
    """

    def __init__(
        self, 
        df_unique: pd.DataFrame, 
        data_root: Path, 
        transform: A.Compose
    ):
        self.df = df_unique.reset_index(drop=True)
        self.data_root = data_root
        self.transform = transform

        self.image_paths = self.df["image_path"].astype(str).values
        
        print(f"📦 データセット作成完了: {len(self)} 画像")

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int) -> Dict:
        """
        インデックスに対応する画像を取得。
        
        Args:
            idx (int): インデックス
        
        Returns:
            Dict: {"image": Tensor, "image_path": str}
        """
        rel_path = self.image_paths[idx]
        img_path = self.data_root / rel_path

        img = Image.open(img_path).convert("RGB")
        img = np.array(img)

        x = self.transform(image=img)["image"]  # torch.Tensor(C,H,W)
        return x, rel_path

## 🚀 推論コア関数¶

### 🔮 単一モデル推論

In [10]:
is_test = True

print()
sep("📝 提出ファイル作成開始")   


# ✅ デバイス設定
# if device is None:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(f"💻 使用デバイス: {device}")

# ✅ 競技ディレクトリ
comp_root = Path("/kaggle/input/csiro-biomass")


# ✅ テストデータ読み込み
if is_test: 
    test_df = pd.read_csv(comp_root / "test.csv")
else:
    test_df = pd.read_csv(comp_root / "train.csv")
print(f"\n📂 テストデータ読み込み中...")
show_df(test_df); print()
# ✅ sample_submissionデータ読み込み
print(f"\n📂 sample_submissionデータ読み込み中...")
sample_sub_df = pd.read_csv(comp_root / "sample_submission.csv"); show_df(sample_sub_df)
print(f"✅ テストデータ読み込み完了: {len(test_df)} 行")

# テストデータのpivot
# ユニークな画像パスを取得
unique_test_df = test_df[["image_path"]].drop_duplicates().reset_index(drop=True)
print(f"🖼️  ユニークな画像数: {len(unique_test_df)}")

# テストデータの前処理
test_df = load_and_prepare_data(df = test_df, is_test = is_test)
print(f"✅ テストデータ前処理完了: {len(test_df)} 行")
show_df(test_df)


# モデル推論
sep("🔮 モデル推論")

for artifact_idx, artifact_root in enumerate(ARTIFACT_ROOTS):
    artifact_root = Path(artifact_root) 
    yaml_path = artifact_root / "yaml" / "config.yaml"
    model_paths = glob_walk((artifact_root) / "model", "*pth")

    # ✅ config読み込み
    cfg = _load_yaml(yaml_path)
    target_cols = cfg["target_cols"]
    print(f"\n📊 設定情報:")
    print(f"   - ターゲット: {target_cols}")
    print(f"   - Log1p変換: {cfg['use_log1p_target']}")


    # transform / loader
    sep("Load transform | data_loader")
    img_size = cfg["img_size"]
    mean = cfg["normalize"]["mean"]
    std = cfg["normalize"]["std"]
    tfm = build_infer_transform(img_size, mean, std)
    ds = TestImageDataset(unique_test_df, data_root=comp_root, transform=tfm)
    test_loader = DataLoader(
        ds,
        batch_size=32,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False,
    )
 
   
    sep("Load models")
    models = []
    for model_path in model_paths:
        print(f"🧠 Load model --{model_path}--")
        # 🧠 モデル構築（pretrainedは必ずFalse）
        model_cfg = cfg["model"]
        model = ConvNeXtRegressor(
            backbone = model_cfg['backbone'],
            pretrained = False,  # ✅ Kaggle対応
            num_targets = len(target_cols),
            in_chans = model_cfg["in_chans"],
            drop_rate= model_cfg["drop_rate"], 
            drop_path_rate= model_cfg["drop_path_rate"],
            head_dropout = 0.0
        ).to(device)
        # チェックポイント読み込み
        ckpt = torch.load(model_path, map_location="cpu", weights_only=False)
        state_dict = _pick_state_dict(ckpt)
        missing, unexpected = model.load_state_dict(state_dict, strict=False)
        if len(missing) > 0:
            print(f"   ⚠️  missing keys: {len(missing)}")
        if len(unexpected) > 0:
            print(f"   ⚠️  unexpected keys: {len(unexpected)}")    
        model.to(device)  # ★ 必ずここで移動
        model.eval()
        models.append(model)

    print(f"\n🧠 --- Finish Load models ---")

    sep("🔮 Inference")
    for model, model_path in zip(models, model_paths):
        model.to(device)
        model.eval()
    
        single_preds = []
        pbar = tqdm(test_loader, desc=f"🔮 推論中: {model_path}")
        with torch.no_grad():
            for images, _ in pbar:
                images = images.to(device)
                pred = model(images)
                single_preds.append(pred.detach().cpu())
    if artifact_idx == 0:
        all_preds = torch.cat(single_preds).numpy()
    else:
        all_preds += torch.cat(single_preds).numpy()


all_preds /= len(ARTIFACT_ROOTS)
print(f"✅ アンサンブル予測完了")


📝 提出ファイル作成開始
💻 使用デバイス: cuda

📂 テストデータ読み込み中...
(5, 3)


,sample_id,image_path,target_name
0,ID1001187975__Dry_Clover_g,test/ID1001187975.jpg,Dry_Clover_g
1,ID1001187975__Dry_Dead_g,test/ID1001187975.jpg,Dry_Dead_g
2,ID1001187975__Dry_Green_g,test/ID1001187975.jpg,Dry_Green_g




📂 sample_submissionデータ読み込み中...
(5, 2)


,sample_id,target
0,ID1001187975__Dry_Clover_g,0.0
1,ID1001187975__Dry_Dead_g,0.0
2,ID1001187975__Dry_Green_g,0.0


✅ テストデータ読み込み完了: 5 行
🖼️  ユニークな画像数: 1
✅ テストデータ前処理完了: 1 行
(1, 7)


...

🔮 推論中: /kaggle/input/csiro-artifacts/exp009/model/best_fold1.pth:   0%|          | 0/1 [00:00<?, ?it/s]

🔮 推論中: /kaggle/input/csiro-artifacts/exp009/model/best_fold2.pth:   0%|          | 0/1 [00:00<?, ?it/s]

🔮 推論中: /kaggle/input/csiro-artifacts/exp009/model/best_fold3.pth:   0%|          | 0/1 [00:00<?, ?it/s]

🔮 推論中: /kaggle/input/csiro-artifacts/exp009/model/best_fold4.pth:   0%|          | 0/1 [00:00<?, ?it/s]

✅ アンサンブル予測完了


In [11]:
# =========================
# Make Submission
# =========================
sep("Check preds")
print(single_preds)
print(all_preds)
print()

# log->raw（学習がlog1pの場合）
if cfg["use_log1p_target"]:
    print(f"🔄 Log1p逆変換を適用中...")
    all_preds = np.expm1(np.clip(all_preds, -20.0, 20.0))

# massは非負なので0未満をクリップ
all_preds = np.clip(all_preds, 0.0, None)
print(f"✂️  負値を0にクリップ")
sep("Check preds")
print(single_preds)
print(all_preds)
print()

# 予測結果を整形
test_df[target_cols] = all_preds
# test_df['GDM_g'] = test_df.Dry_Green_g + test_df.Dry_Clover_g
# test_df['Dry_Total_g'] = test_df.GDM_g + test_df.Dry_Dead_g
sep("pred_result"); show_df(test_df)

Check preds
[tensor([[3.3374, 0.4959, 3.7697, 3.5374, 3.9095]])]
[[3.3373547  0.49588552 3.769688   3.5374036  3.9095285 ]]

🔄 Log1p逆変換を適用中...
✂️  負値を0にクリップ
Check preds
[tensor([[3.3374, 0.4959, 3.7697, 3.5374, 3.9095]])]
[[27.144577    0.64195156 42.366528   33.377544   48.87543   ]]

pred_result
(1, 7)


,sample_id_prefix,image_path,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g
0,ID1001187975,test/ID1001187975.jpg,0.641952,42.366528,27.144577,48.875431,33.377544


In [12]:
# =========================
# Make Submission
# =========================

sub_df = test_df.set_index('sample_id_prefix')[target_cols].stack()
sub_df = sub_df.reset_index()
sub_df.columns = ['sample_id_prefix', 'target_name', 'target']
sub_df['sample_id'] = sub_df.sample_id_prefix + '__' + sub_df.target_name
show_df(sub_df, 5)

submission_cols = ['sample_id', 'target']
sub_df[submission_cols].to_csv('submission.csv', index=False)

print("✅ submission.csv 作成完了")
print("\n📄 提出ファイルプレビュー:")
show_df(sub_df[submission_cols], 5)

sep("🎉 全処理完了!")


(5, 4)


,sample_id_prefix,target_name,target,sample_id
0,ID1001187975,Dry_Green_g,27.144577,ID1001187975__Dry_Green_g
1,ID1001187975,Dry_Clover_g,0.641952,ID1001187975__Dry_Clover_g
2,ID1001187975,Dry_Dead_g,42.366528,ID1001187975__Dry_Dead_g
3,ID1001187975,GDM_g,33.377544,ID1001187975__GDM_g
4,ID1001187975,Dry_Total_g,48.875431,ID1001187975__Dry_Total_g


✅ submission.csv 作成完了

📄 提出ファイルプレビュー:
(5, 2)


,sample_id,target
0,ID1001187975__Dry_Green_g,27.144577
1,ID1001187975__Dry_Clover_g,0.641952
2,ID1001187975__Dry_Dead_g,42.366528
3,ID1001187975__GDM_g,33.377544
4,ID1001187975__Dry_Total_g,48.875431


🎉 全処理完了!


In [13]:
sub_df.describe()

,target
count,5.000000
mean,30.481207
std,18.638075
min,0.641952
25%,27.144577
50%,33.377544
75%,42.366528
max,48.875431
